# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
# import libraries
import os
import sys
import pathlib

cwd = os.getcwd()

# get the parent directory
parent_dir = os.path.dirname(cwd)

# add the parent directory to the system path
sys.path.insert(0, parent_dir)

# Get to the root directory
project_root = pathlib.Path().absolute().parent

# import functions from common like this:
from pipeline.common.model_training import (
    training_config as tc,
    modelling_functions as mf,
    model_properties as mp
)

from pipeline.common.model_prediciton import prediction_functions as pf

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [ ]:
training_data = mf.get_training_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/training_data.sql')

training_data

## Modelling
During the modelling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of three distinct models: XGBoost, Random Forest, and Gradient Boosting Classifier. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best model, or `footy_tipper`, is selected based on the superior performance on the chosen optimization metric. Additionally, a `LabelEncoder`(`label_encoder`), used to encode the categorical target variable, is returned. This LabelEncoder is specific to the model that performed best. The selected model, encapsulated in a pipeline with pre-processing steps and hyperparameter tuning, is now ready for the prediction phase.

In [ ]:
footy_tipper, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)
footy_tipper

### Display feature importance
The `get_feature_importance` function retrieves feature importances from a trained scikit-learn pipeline. It accounts for different transformations, such as one-hot encoding and recursive feature elimination. The function then returns a sorted DataFrame listing each feature alongside its respective importance, aiding in understanding the model's decision-making process.

In [ ]:
# feature_importance_df = mp.get_feature_importances_from_pipeline(footy_tipper, tc.predictors)
# feature_importance_df

## Save Model
The `save_models` function stores the trained LabelEncoder and Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [ ]:
mf.save_models(label_encoder, footy_tipper, project_root)

## Predict
The final stage of the pipeline involves predicting the outcomes of the current week's NRL matches. This is achieved by connecting to the SQLite database and extracting the required data. The trained model and LabelEncoder are then loaded from the disk, and the prediction is performed using the `model_predictions` function. The predictions are stored in the 'predictions' table of the database, allowing for easy retrieval and analysis.

In [2]:
label_encoder, footy_tipper = pf.load_models(project_root)

Loading models...
Label encoder loaded
Model pipeline loaded


In [3]:
inference_data = pf.get_inference_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/inference_data.sql')
inference_data

Getting inference data...


,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.024111e+10,5.0,Round 5,1.0,Pre Game,1.712261e+09,1.712221e+09,AAMI Park,Melbourne,None,...,22.0,-18.0,1516.275208,1528.854362,-12.579154,0.420501,0.445364,0.134135,-0.024862,8.787267
1,2.024111e+10,5.0,Round 5,2.0,Pre Game,1.712340e+09,1.712300e+09,Accor Stadium,Sydney,None,...,7.0,25.0,1472.301266,1525.311750,-53.010484,0.415486,0.555370,0.029144,-0.139884,-7.525067
2,2.024111e+10,5.0,Round 5,3.0,Pre Game,1.712347e+09,1.712308e+09,McDonald Jones Stadium,Newcastle,None,...,38.0,-36.0,1502.365709,1477.315509,25.050199,0.525145,0.447871,0.026984,0.077274,4.923967
3,2.024111e+10,5.0,Round 5,4.0,Pre Game,1.712416e+09,1.712376e+09,Accor Stadium,Sydney,None,...,4.0,0.0,1475.771727,1510.689861,-34.918134,0.440404,0.530452,0.029144,-0.090048,4.500633
4,2.024111e+10,5.0,Round 5,5.0,Pre Game,1.712425e+09,1.712385e+09,4 Pines Park,Sydney,None,...,-6.0,13.0,1501.744703,1542.805907,-41.061204,0.431910,0.538946,0.029144,-0.107037,-6.907100
5,2.024111e+10,5.0,Round 5,6.0,Pre Game,1.712428e+09,1.712392e+09,Suncorp Stadium,Brisbane,None,...,-1.0,39.0,1501.407899,1482.035475,19.372424,0.517235,0.455781,0.026984,0.061454,6.524433
6,2.024111e+10,5.0,Round 5,7.0,Pre Game,1.712506e+09,1.712470e+09,Queensland Country Bank Stadium,Townsville,None,...,32.0,-31.0,1508.435844,1455.538398,52.897445,0.571955,0.415545,0.012500,0.156410,14.875600
7,2.024111e+10,5.0,Round 5,8.0,Pre Game,1.712514e+09,1.712478e+09,GIO Stadium,Canberra,None,...,8.0,12.0,1494.308957,1500.782212,-6.473255,0.428106,0.437758,0.134135,-0.009652,7.174133


In [4]:
predictions_df = pf.model_predictions(footy_tipper, inference_data, label_encoder)
predictions_df

Making predictions...


,game_id,home_team_result,home_team_win_prob,home_team_lose_prob
0,2.024111e+10,Win,0.673769,0.326231
1,2.024111e+10,Loss,0.265748,0.734252
2,2.024111e+10,Win,0.667742,0.332258
3,2.024111e+10,Loss,0.366799,0.633201
4,2.024111e+10,Loss,0.291297,0.708703
5,2.024111e+10,Win,0.750245,0.249755
6,2.024111e+10,Win,0.870425,0.129575
7,2.024111e+10,Win,0.657059,0.342941


In [5]:
pf.save_predictions_to_db(
    predictions_df, 
    project_root / "data" / "footy-tipper-db.sqlite", 
    project_root / 'pipeline/common/sql/create_table.sql', 
    project_root / 'pipeline/common/sql/insert_into_table.sql'
)

Saving predictions to database...


# this is the sending bit

In [6]:
from dotenv import load_dotenv
from pipeline.common.model_prediciton import prediction_functions as pf
from pipeline.common.use_predictions import sending_functions as sf

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"
json_path = project_root / "service-account-token.json"

load_dotenv(dotenv_path=secrets_path)

True

In [7]:
import sqlite3
import pandas as pd
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open(project_root / 'pipeline/common' / 'sql/prediction_table.sql', 'r') as file:
    query = file.read()

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

predictions

,game_id,home_team_result,team_home,position_home,team_head_to_head_odds_home,team_away,position_away,team_head_to_head_odds_away,home_team_win_prob,home_team_lose_prob,round_id,competition_year,round_name
0,20241110510,Win,Melbourne Storm,5,1.45,Brisbane Broncos,12,2.77,0.673769,0.326231,5,2024,Round 5
1,20241110520,Loss,Canterbury-Bankstown Bulldogs,14,4.00,Sydney Roosters,7,1.25,0.265748,0.734252,5,2024,Round 5
2,20241110530,Win,Newcastle Knights,15,1.47,St. George Illawarra Dragons,13,2.70,0.667742,0.332258,5,2024,Round 5
3,20241110540,Loss,South Sydney Rabbitohs,16,2.12,New Zealand Warriors,10,1.73,0.366799,0.633201,5,2024,Round 5
4,20241110550,Loss,Manly-Warringah Sea Eagles,11,2.31,Penrith Panthers,2,1.62,0.291297,0.708703,5,2024,Round 5
5,20241110560,Win,Dolphins,1,1.54,Wests Tigers,6,2.48,0.750245,0.249755,5,2024,Round 5
6,20241110570,Win,North Queensland Cowboys,3,1.14,Gold Coast Titans,17,5.80,0.870425,0.129575,5,2024,Round 5
7,20241110580,Win,Canberra Raiders,8,1.78,Parramatta Eels,9,2.08,0.657059,0.342941,5,2024,Round 5


In [8]:
tipper_picks = sf.get_tipper_picks(predictions)
tipper_picks

,team,price,price_min
5,Dolphins,1.54,1.332898
7,Canberra Raiders,1.78,1.521933
3,New Zealand Warriors,1.73,1.579277
4,Penrith Panthers,1.62,1.411028


In [9]:
sf.upload_df_to_drive(
    predictions, 
    json_path, 
    os.getenv('FOLDER_ID'), 
    "predictions.csv"
)

File ID: 1ZNQoeEKLo4Wei0c3atK3fCi_A90Y-fjy


In [10]:
reg_reagan = sf.generate_reg_regan_email(
    predictions, 
    tipper_picks, 
    os.getenv('OPENAI_KEY'), 
    os.getenv('FOLDER_URL'),
    1
)

print(reg_reagan)

Subject: Round 5 NRL Predictions & The Best Bets - Biff Included 🏉 💰

Hey Mates, 

Reg Reagan here, shooting through your absolute must-read synopsis for Round 5 of the NRL. Get ready to throw some banter around and make sure you’re currently sitting, because I’ve got some gold for you lot. 

Firstly, let's cut to the chase with the Footy Tipper predictions – your new bible for the betting season. 

🔮 **Match Highlights:**

- **Melbourne Storm** vs. Brisbane Broncos: Storm’s gonna cook up a storm, sitting pretty at 5th, facing off 12th placed Broncos. Bookies have them at a steal for 1.45. That’s practically robbery without wearing a mask.

- Bulldogs are about to play dead again, hosting the **Roosters**. At 14th, we could probably see more action in a retirement village. Roosters at 1.25, because let's face it, it's almost a guaranteed win.

- **Knights vs. Dragons**: As a Knights tragic, I’m calling it – a win for the boys! Don’t let that 15th position fool you; we’re climbing faste

In [11]:
sf.send_emails(
    "footy-tipper-email-list", 
    f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}", 
    reg_reagan, 
    os.getenv('MY_EMAIL'), 
    os.getenv('EMAIL_PASSWORD'), 
    json_path
)